In [1]:
import os
import sys
import glob
import obspy
sys.path.append('..')
import setup_paths
paths = setup_paths.paths
sys.path.append('../../src/lib')
from SAM import RSAM, VSEM

/home/thompsong/work/PROJECTS/MESS2024_Glenn exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SDS exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SDS_VEL exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SDS_DISP exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/binary exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/responses exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/events exists
/home/thompsong/work/PROJECTS/MESS2024_Glenn/db exists


In [2]:
startTime = obspy.core.UTCDateTime(2001,7,29,18,0,0)
endTime = obspy.core.UTCDateTime(2001,7,29,23,59,59)
secondsPerDay = 60 * 60 * 24
numDays = (endTime-startTime)/secondsPerDay

In [3]:
rsamMV = VSEM.read(startTime, endTime, SAM_DIR=paths['SAM_DIR'], sampling_interval=10)
print(rsamMV)
rsamMV_Z = rsamMV.select(component='Z')
#rsamMV_Z.plot(metrics=['energy'])

/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_*_2001_10s.pickle
['/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBGB..BHN_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBLG..SHZ_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBBY..BHZ_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBMH..SHN_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBWH..SHZ_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBRY..BHN_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBGH..BHN_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBBY..BHE_2001_10s.pickle', '/home/thompsong/work/PROJECTS/MESS2024_Glenn/data/continuous/SAM/VSEM_MV.MBGH..BHZ_2001_10s.pi

In [4]:
from obspy.core.inventory.inventory import read_inventory
inv = read_inventory(os.path.join(paths['RESPONSE_DIR'],'MV.xml'), format='stationxml')  
source = {'lat':16.71111, 'lon':-62.17722}
rsamMV_Z.sum_energy(startTime, endTime, inv, source)

MV.MBBY..BHZ 1.353656040247293e-26 -20.378994449665555
MV.MBWH..SHZ 4968840524.512485 3.3308367054931094
MV.MBMH..SHZ 7402659657.831103 3.446258521910287
MV.MBLG..SHZ 68225093000.5664 4.089296090898582
MV.MBRY..BHZ 10834847049.841825 3.5565485233196394
MV.MBGB..BHZ 8636272073.79343 3.490884210482678
MV.MBGH..BHZ 5934081906.224186 3.3822356909661733
MV.MBSS..SHZ 2269237955.911333 3.1039200260565902


In [ ]:
# drop IDs that don't look good
rsamMV_Z.drop('MV.MBBY..BHZ')
rsamMV_Z.drop('MV.MBMV..EHZ')

# downsample from 1-minute to 10-minutes
rsamMV_Zhourly = rsamMV_Z.downsample(new_sampling_interval=600)

# plot mean & median, then LP and VT
rsamMV_Z.plot(metrics=['mean', 'median'], kind='line')
rsamMV_Zhourly.plot(metrics=['LP', 'VT'], kind='line')

In [ ]:
rsamMV.plot(metrics=['specratio'], equal_scale=True)

# Exercise 1: make this exercise 2? put Montserrat example here?
For this exercise, we are going to plot RSAM data recorded by the original RSAM system installed at Montserrat Volcano Observatory (MVO) that was installed in July 1995 by VDAP scientists. Specifically, we will look at data from summer 1996, which was my first time at MVO.

## 1.1 First we do some imports, and set some constants

In [ ]:
import os
import sys
sys.path.append('../../lib')
import FLOVO
import obspy
RSAM_DIR = '../../data/continuous/RSAM'
RSAM_BINARY_DIR = os.path.join(RSAM_DIR, 'binary')

## 1.2. Convert RSAM binary files
The original RSAM system recorded 1-minute RSAM data in binary files with a 4-byte float for each sample, for every minute of the year. 
Let us convert these files so we never have to bother with these binary files again. So we will read the whole year of 1996.

In [ ]:
stime = obspy.core.UTCDateTime(1996,1,1,0,0,0)
etime = obspy.core.UTCDateTime(1996,12,31,23,59,59)
stations = ['MCPZ', 'MGAT', 'MGHZ', 'MLGT', 'MRYT', 'MSPT', 'MSSZ', 'MWHT']
rsamObj = FLOVO.readRSAMbinary(RSAM_BINARY_DIR, stations, stime, etime)
#rsamObj.write(RSAM_DIR, ext='csv')

The MVO network in 1996 didn't use modern SEED naming convention (network.station.location.channel or 'NSLC'), so here a network of 'MV' had been added, and the channel name has been set to 'EHZ' where 'E' indicates the original data were recorded at 100 Hz sampling rate, 'H' indicates seismic data, and 'Z' indicates a vertical component seismic sensor. 

'60s' has also been appended to the filename to indicate the sampling interval for the RSAM data. 

In memory, data for each station is stored in a pandas DataFrame, and on disk as a CSV file, which allows for easy browsing. 

To illustrate, next we will:

- print the RSAM object
- print the pandas DataFrame corresponding to id='MV.MLGT..EHZ'
- inspect the corresponding CSV file
- plot() the RSAM object

In [ ]:
print('RSAM object:')
print(rsamObj)systems

print('\nDataFrame:')
print(rsamObj.dataframes['MV.MLGT..EHZ'].head())

print('\nCSV file:')
os.system('head ../../data/continuous/RSAM/RSAM_MV.MLGT..EHZ_1996_60s.csv')


Each of these provides similar information. There are just two columns, which are 'time', and 'mean'. 
- 'time' is in Unix epoch seconds (since 1970-01-01 00:00:00)
- 'mean' just holds the mean seismic amplitude within that 60-s time window (Sampling Interval=60.0s)

# 1.3 Reading a subset of the data we just saved, and plotting an RSAM object
Next we will re-read (from disk) the RSAM data from 1996-02-15 to 1996-10-12, and then plot the data. By default, the plot() method will convert RSAM dataframes into an ObsPy Stream object, so it can be plotted in a familiar way. 

In [ ]:
startt = obspy.core.UTCDateTime(1996,2,15)
endt = obspy.core.UTCDateTime(1996,10,12)
rsamObj = FLOVO.readRSAMfile(startt, endt, trace_ids=['MV.MLGT..EHZ', 'MV.MGAT..EHZ', 'MV.MRYT..EHZ', 'MV.MGHZ..EHZ'], RSAM_TOP=RSAM_DIR, ext='csv')
rsamObj.plot()   

These RSAM plots above show the following general features:
1. Low seismicity in February and March.
2. An increase in seismicity around April 1st persists throughout to June. This period included the first pyroclastic density current (PDC) that reached the ocean on May 12, 1996.
3. A more significant increase in activity about 2/3rds of the way through July 1996. This was a time period in which the seismicity and the lava dome extrusion rate significantly increased, leading to numerous PDCs that reached the ocean, and even travelled for some distance upon the water. The increase is particularly noticeable on MV.MLGT..EHZ (3rd trace) as this was close to the Tar River Valley, where most PDCs were directed.
4. A sharp drop in seismicity from September 18, 1996, onwards.

These features may be more obvious if we smooth the data, which we can do with the RSAMmetrics.smooth() method:

In [ ]:
print('rsamObj:\n',rsamObj)
rsamObjHourly = rsamObj.copy()
rsamObjHourly.smooth(binsize=3600) 
print('rsamObjHourly:\n',rsamObjHourly)
rsamObjHourly.plot()

## 1.4 Trim, and Plot 1-minute RSAM data in summer 1996

Let us zoom in on the hourly data we smoothed earlier. We can do this with the trim() method, followed by plot().
trim() is done inplace, i.e. the original RSAM object is replaced.


In [ ]:
startt = obspy.core.UTCDateTime(1996,7,15)
endt = obspy.core.UTCDateTime(1996,9,1)
rsamObjHourly.trim(starttime=startt, endtime=endt)
rsamObjHourly.plot(kind='stream', equal_scale=False)   

There are various periods here where there seem to by cycles in RSAM. Let us look at early August period in more detail.


In [ ]:
rsamObjMidJuly = rsamObj.copy()
rsamObjMidJuly.trim(starttime=obspy.core.UTCDateTime(1996,8,1), endtime=obspy.core.UTCDateTime(1996,8,8))
rsamObjMidJuly.plot(kind='stream', equal_scale=False) 

These are remarkable cycles in RSAM. They appear to be about 4-6 hours apart. This is a phenomenon called "banded tremor". During these tremor bands, visual observations indicated that the lava dome was extruding at particularly high rates (up to 20m^3 was one estimate I heard), and at the peak of each cycle there was often ash venting. I proposed that the tremor bands were indicated of pressure cycles within the conduit - but caused by what? 
One suggestion is that the magma rises up the conduit in a stick-slip fashion. Basically, it gets stuck for a while, as the pressure builds below, and then shear fractures, allowing magma to suddenly extrude very quickly. 

Can we use some ObsPy STA/LTA detection tools to detect these tremor bands, in the same way we normally detect much shorter transient events, but just with longer STA/LTA settings? Let us try first on a single NSLC. This is based on examples at https://docs.obspy.org/tutorial/code_snippets/trigger_tutorial.html, except we use longer STA and LTA time windows (15 and 100 minutes respectively), and we add a despiking step which attempts to remove transient events lasting a minute or less from the data before running the STA/LTA:



In [ ]:
from obspy.signal.trigger import plot_trigger, classic_sta_lta, recursive_sta_lta
st = rsamObjMidJuly._to_stream()
st.trim(obspy.core.UTCDateTime(1996,8,3), obspy.core.UTCDateTime(1996,8,5))

def despike(st, thresh=1.5):
    for tr in st:
        x = tr.data
        for i in range(len(x)-2):
            if x[i+1]>x[i]*thresh and x[i+1]>x[i+2]*thresh:
                x[i+1] = (x[i] + x[i+2])/2
        tr.data = x
despike(st)
trace = st[2].copy()
trace.plot();

sta_minutes = 15
lta_minutes = 100
threshON = 1.0
threshOFF = 0.3

cft = recursive_sta_lta(trace.data, sta_minutes, lta_minutes)

plot_trigger(trace, cft, threshON, threshOFF)

That seems to work quite well. Now let us try an event detector that uses several NSLC at once.

In [ ]:
from obspy.signal.trigger import coincidence_trigger
from pprint import pprint
import numpy as np

threshStations = 3

trig = coincidence_trigger("recstalta", threshON, threshOFF, st, threshStations, sta=sta_minutes*60, lta=lta_minutes*60, max_trigger_length=2*lta_minutes*60, delete_long_trigger=True)

#pprint(trig)

lendata = len(st[0].data)
trdata = np.zeros( (lendata, ) )
detectionTrace = obspy.Trace( data = trdata ) 
detectionTrace.id = 'XX.DETEC..TED'
detectionTrace.stats.starttime = st[0].stats.starttime
detectionTrace.stats.sampling_rate = st[0].stats.sampling_rate
t = detectionTrace.times('utcdatetime')
for thistrig in trig:
    t0 = thistrig['time']
    t1 = (thistrig['time'] + thistrig['duration'])
    indices = np.where((t >= t0) & (t <= t1))
    #print(t0, t1, indices)
    detectionTrace.data[indices] = 1 #thistrig['duration']

st3 = st.copy()
st3.append(detectionTrace)
st3.plot(equal_scale=False);

detection_ON_times = [thistrig['time'].timestamp for thistrig in trig]
detection_intervals_minutes = np.diff(np.array(detection_ON_times))/60
for i,d in enumerate(detection_intervals_minutes):
    print(f"detection ON time for band {i}: {trig[i]['time']}, duration: {trig[i]['duration']/60} mins")
    print(f"- interval (mins): {detection_intervals_minutes[i]}")
print(f"detection ON time for band {i+1}: {trig[i+1]['time']}")

The bottom trace here corresponds to the detected events, and you can see they line up pretty well with the tremor bands, except the first one was missed.

This is similar to the banded tremor alarm system I wrote at MVO in 2000. And using this approach we can forecast the timing of the next tremor band. As it was the MVO Seismologist's job to manage the Operations Room, which included continuous seismic monitoring and two-way radio communications with MVO field crews, it was useful to predict tremor bands, as these were periods of heightened activity when field crews should not be on the flanks of the volcano.




In [ ]:
# further isolate bands by stacking and applying detection mask
stackedTrace = st3[0].copy()
stackedTrace.id = 'MV.STACK..EHZ'
for tr in st3[1:-1]:
    stackedTrace.data += tr.data
stackedTrace.plot();

maskedTrace = stackedTrace.copy()
#maskedTrace.stats.station = 'MASK'
#maskedTrace.data *= st3[-1].data

# find peak value and peak time during each band
import pandas as pd
lod = []
for thistrig in trig:
    bandstarttime = thistrig['time']
    bandendtime = thistrig['time'] + thistrig['duration']
    bandTrace = maskedTrace.copy().trim(starttime=bandstarttime, endtime=bandendtime)
    bandpeaktime = bandstarttime + bandTrace.data.argmax() * tr.stats.delta
    band = {'starttime':bandstarttime, 'waxtime':bandpeaktime-bandstarttime, \
            'peaktime':bandpeaktime, 'wanetime':bandendtime-bandpeaktime, 'endtime':bandendtime, 'duration':thistrig['duration']}
    lod.append(band)


bandDf = pd.DataFrame(lod)
print(bandDf)

predicted = []
for col in ['starttime', 'peaktime', 'endtime']:
    interval = (bandDf.iloc[-1][col] - bandDf.iloc[0][col]) / (len(bandDf)-1) 
    predicted.append(bandDf.iloc[-1][col] + interval)
print('\nNext band prediction:')
print(' - start: ',predicted[0])
print(' - peak:  ',predicted[1])
print(' - end:   ',predicted[2])

st5 = rsamObjMidJuly.copy()
st5.trim(starttime = obspy.core.UTCDateTime(1996,8,5,0,0,0), endtime = obspy.core.UTCDateTime(1996,8,5,3,0,0) )
st5.plot()

# Exercise 2: 
For this exercise, we are going to use continuous seismic data from Montserrat in late February/early March, 2001, and compute our own RSAM data.

# 2.2 Redoubt Volcano, Alaska, 2009

In [ ]:
# Now compute for Redoubt
stime = obspy.core.UTCDateTime(2009,3,22,22,0,0)
etime = obspy.core.UTCDateTime(2009,3,23,8,0,0)
from obspy.clients.filesystem.sds import Client as SDSCLIENT
myclient = SDSCLIENT('../../data/continuous/SDS')
stRD = myclient.get_waveforms("AV", "*", "*", "[HDESB]*", stime, etime)
#print(stRD.__str__(extended=True))

In [ ]:
# attempt to merge traces by rounding off the sampling rate
stRD.sort(['starttime', 'endtime', 'npts', 'network', 'station', 'location', 'channel'])
stRD2 = stRD.copy()
try:
    stRD2.merge(method=0, fill_value='interpolate')
except:
    stRD2 = obspy.core.Stream()
    for tr in stRD:
        try:
            tr.stats.sampling_rate=round(tr.stats.sampling_rate)
            tr.data[abs(tr.data)>10000]=0
            stRD2.append(tr)
            stRD2.merge(method=0, fill_value='interpolate')
            #print(tr)
        except:
            print(f"Cannot merge {tr}")
stRD2.plot(starttime=stime+4.8*3600, endtime=stime+5*3600, equal_scale=False);

In [ ]:
good_ids = ['AV.RDN..EHZ', 'AV.REF..EHZ', 'AV.REF..EHN', 'AV.REF..EHE', 'AV.RSO..EHZ', 'AV.RDWB..BHZ', 'AV.RDWB..BHN', 'AV.RDWB..BHE']
stRD3 = obspy.core.Stream()
for id in good_ids:
    stRD3.append(stRD2.select(id=id)[0])
importlib.reload(FLOVO)
r2 = FLOVO.RSAMmetrics(stream=stRD3, sampling_interval=60, filter=True)
r2.plot(kind='stream', metrics=['mean'])

In [ ]:
r2.plot(kind='stream', metrics=['specratio'])

In [ ]:
stRD4 = stRD3.copy().trim(starttime=stime+4.8*3600, endtime=stime+5*3600)
importlib.reload(FLOVO)
r3 = FLOVO.RSAMmetrics(stream=stRD4, sampling_interval=3, filter=True)
r3.plot(kind='stream');

## 2.3 Soufriere Hills Volcano, Montserrat, 2001

In [ ]:
import os
import sys
import obspy
from obspy.clients.filesystem.sds import Client as sdsclient

PROJECT_DIR = os.path.join('..','..')
LIB_DIR = os.path.join(PROJECT_DIR,'lib')
sys.path.append(LIB_DIR)
import FLOVO

CONTINUOUS_DATA_DIR = os.path.join(PROJECT_DIR, 'data', 'continuous')
SDS_DIR = os.path.join(CONTINUOUS_DATA_DIR, 'SDS')
RSAM_DIR = os.path.join(CONTINUOUS_DATA_DIR, 'RSAM')

#####################################################################################

mySDSclient = sdsclient(SDS_DIR)
startTime = obspy.core.UTCDateTime(2001,2,25,0,0,0)
endTime = obspy.core.UTCDateTime(2001,3,5,0,0,0)
secondsPerDay = 60 * 60 * 24
numDays = (endTime-startTime)/secondsPerDay

In [ ]:
tr2 = obspy.core.Trace(data=np.array(rsam_values))
tr2.stats.starttime = stime
tr2.stats.delta = time_window
tr2.id = tr.id
tr2.stats.channel = 'U' + tr.stats.channel[1:]

st = obspy.core.Stream([tr, tr2])
st.plot(equal_scale=False);

In [ ]:
print('median ',np.median(tr.data))
print('max ',np.max(tr.data))
print('std ',np.std(tr.data))
print('99th percentile ',np.percentile(tr.data, 99))

In [ ]:
t = trMBWH.times() # seconds from start of seismogram
udt = trMBWH.stats.starttime
time_window = 60 # 60-s time window
rsam_values = []
rsam_times = []
for startt in range(t[0], t[-1], time_window):
    tr_segment = trMBWH.copy().trim(starttime=udt+startt, endtime=udt+startt+time_window)
    m = np.mean(tr_segment.data)
    rsam_times.append(u+startt+time_window)
    rsam_values.append(m)

import matplotlib.pyplot as plt
plt.plot(rsam_times.datetime, rsam_values)


In [ ]:
daytime = startTime
while daytime < endTime:
    print(f'Loading Stream data for {daytime}')
    stMV = mySDSclient.get_waveforms("MV", "*", "*", "[SBEHCD]*", daytime, daytime+secondsPerDay)

    print(f'Computing RSAM metrics for {daytime}, and saving to CSV')
    rsamMV24h = FLOVO.RSAM(stream=stMV, sampling_interval=60)
    rsamMV24h.write(RSAM_DIR)
    
    daytime += secondsPerDay

del mySDSclient

print(f'Reading and Plotting {numDays} days of RSAM metrics')


rsamMV = FLOVO.readRSAMfile(startTime, endTime, trace_ids=None, RSAM_TOP=RSAM_DIR)
print(rsamMV)
rsamMV.plot(kind='stream', metrics=['median'])

In [ ]:
# add frequency index to RSAMmetrics
# add a 1/rsam plot for FFM
# add a way to compute RSAM for overlapping time bins - easy to do if say select 1 minute first, and then apply say a 10-point moving point filter